In [14]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from urllib.parse import urljoin
from io import BytesIO
import cv2

In [15]:
# search_terms = ["underbench oven", "kitchen exhaust", "kitchen island", "double sink"]
search_terms = ["underbench oven", "double sink on counter"]
num_images = 10
save_folder = "raw_images"

In [13]:
## run this to start downloading images from google
def save_google_images(search_terms, num_images, save_folder):
    # Create the save folder if it doesn't exist
    os.makedirs(save_folder, exist_ok=True)

    for search_term in search_terms:
        # Create a folder for the current search term
        search_folder = os.path.join(save_folder, search_term)
        os.makedirs(search_folder, exist_ok=True)
        filename_number = len(os.listdir(os.path.join(save_folder, search_term)))


        # Prepare the search URL
        search_url = f"https://www.google.com/search?q={search_term}&tbm=isch"

        # Send a GET request to the search URL
        response = requests.get(search_url)
        response.raise_for_status()

        # Parse the response content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all image elements on the page
        image_elements = soup.find_all('img')

        # Iterate over the image elements and save the thumbnails
        count = 0
        for img in image_elements:
            if count == 0:
                count+=1
                filename_number += 1
                continue
            # Get the thumbnail URL
            thumbnail_url = img['src']
            if thumbnail_url.startswith('/'):
                # Handle relative URLs by joining with the base URL
                thumbnail_url = urljoin(search_url, thumbnail_url)

            # Send a GET request to the thumbnail URL
            thumbnail_response = requests.get(thumbnail_url)
            thumbnail_response.raise_for_status()

            # Open the thumbnail image with PIL
            image = Image.open(BytesIO(thumbnail_response.content))

            # Convert the image to RGB mode
            image = image.convert("RGB")

            # Resize the image to the desired size
            image = image.resize((150, 150))

            # Save the resized image in the search term folder
            filename = f"{search_folder}/{filename_number}_{search_term}.jpg"
            image.save(filename)

            count += 1
            filename_number += 1
            if count == num_images:
                break

        print(f"{count} thumbnail images saved for '{search_term}' in {search_folder}")


# Example usage
save_google_images(search_terms, num_images, save_folder)

10 thumbnail images saved for 'underbench oven' in raw_images\underbench oven
10 thumbnail images saved for 'double sink on counter' in raw_images\double sink on counter


In [26]:
boxing_terms = ["underbench oven"]

In [27]:
# Run to draw bounding boxes
boxed_images = './boxed_images'

if not os.path.exists(boxed_images):
    os.makedirs(boxed_images)

for appliancename in boxing_terms:
    output_folder_path = boxed_images + '/' + appliancename

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    input_folder_path = os.path.join('raw_images', appliancename)

    for filename in os.listdir(input_folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Update file extensions as needed
            # Load the input image
            image_path = os.path.join(input_folder_path, filename)
            image = cv2.imread(image_path)

            # Create a copy of the image to draw the bounding box
            annotated_image = image.copy()

            # Create a window and display the image
            cv2.namedWindow("Image")
            cv2.imshow("Image", annotated_image)

            # Create variables to store the coordinates of the bounding box
            bbox_start = None
            bbox_end = None
            drawing = False

            # Mouse callback function to draw the bounding box
            def draw_bbox(event, x, y, flags, param):
                global bbox_start, bbox_end, drawing

                if event == cv2.EVENT_LBUTTONDOWN:
                    bbox_start = (x, y)
                    drawing = True
                elif event == cv2.EVENT_LBUTTONUP:
                    bbox_end = (x, y)
                    drawing = False
                    cv2.rectangle(annotated_image, bbox_start, bbox_end, (0, 0, 255), 2)
                    cv2.imshow("Image", annotated_image)

            # Set the callback function for mouse events
            cv2.setMouseCallback("Image", draw_bbox)

            # Wait for the user to draw the bounding box and press 'Enter' to continue
            while True:
                key = cv2.waitKey(1) & 0xFF
                if key == 13:  # 'Enter' key
                    break
                elif key == 8:  # 'Backspace' key
                    bbox_start = None
                    bbox_end = None
                    annotated_image = image.copy()
                    cv2.imshow("Image", annotated_image)

            # Save the annotated image with the bounding box to the output folder
            output_image_path = os.path.join(output_folder_path, filename)
            cv2.imwrite(output_image_path, annotated_image)

            # Close the window
            cv2.destroyAllWindows()